In [6]:
import numpy as np
import pandas as pd

In [54]:
def aggregated_directional_change(
    prices: pd.DataFrame,
    w_min: int = 1,
    w_max: int = 50,
    alpha_func=None,
    smooth_func=None,
    epsilon: float = 1e-8
) -> np.ndarray:
    """
    Вычисляет Aggregated Directional Change с симметричными окнами:
    (x_{t+w} - x_{t-w}) / (x_t + ε)
    """
    prices = df["price"].values
    n = len(prices)

    if alpha_func is None:
        alpha_func = lambda w: 1.0 / w
    if smooth_func is None:
        smooth_func = lambda x: x

    ws = np.arange(w_min, w_max + 1)
    weights = np.array([alpha_func(w) for w in ws])

    adc = np.full(n, np.nan)

    for i, w in enumerate(ws):
        left = np.roll(prices, w)
        right = np.roll(prices, -w)
        center = prices

        # Вырезаем края, чтобы не было артефактов от np.roll
        valid_range = np.arange(w, n - w)

        delta = (right[valid_range] - left[valid_range]) / (center[valid_range] + epsilon)
        smoothed = smooth_func(delta)
        weighted = weights[i] * smoothed

        if i == 0:
            acc = np.zeros_like(prices, dtype=np.float64)
        acc[valid_range] += weighted

    # Оставим значения только в валидной зоне (вокруг которых были данные)
    adc[w_max: n - w_max] = acc[w_max: n - w_max]

    return adc

In [160]:
w = 100

input_file = "../data/prices/BTC_1000ms.csv"
output_file = f"../data/adc/BTC_1000ms_w{w}.csv"

df = pd.read_csv(input_file)
adc = aggregated_directional_change(
    df,
    w_min=1,
    w_max=w,
    alpha_func=lambda w: 1 / w ** (1/2),
    smooth_func=lambda x: np.tanh(5 * x)
)

In [161]:
pd.Series(adc).to_csv(output_file, header=["adc"], index=False)